# Método Monte Carlo (Parte II)

In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import math
import scipy.stats as stats
import random

## Generamos el Dataframe de Entradas

In [2]:
activities = {
    "N": [1, 2, 3, 4, 5, 6, 7, 8],
    "Actividades": ["Actividad 1", "Actividad 2", "Actividad 3", "Actividad 4", "Actividad 5", "Actividad 6", "Actividad 7", "Actividad n"],
    "Min": [36, 27, 72, 45, 90, 63, 54, 18],
    "+Probable": [40, 30, 80, 50, 100, 70, 60, 20],
    "Max": [50, 37.5, 100, 62.5, 125, 87.5, 75, 25],
    "Ruta_Critica": [1, 0, 1, 0, 1, 0, 0, 1],
}

activities_df = pd.DataFrame(activities)

print(activities_df)

   N  Actividades  Min  +Probable    Max  Ruta_Critica
0  1  Actividad 1   36         40   50.0             1
1  2  Actividad 2   27         30   37.5             0
2  3  Actividad 3   72         80  100.0             1
3  4  Actividad 4   45         50   62.5             0
4  5  Actividad 5   90        100  125.0             1
5  6  Actividad 6   63         70   87.5             0
6  7  Actividad 7   54         60   75.0             0
7  8  Actividad n   18         20   25.0             1


In [3]:
activities_df['Media_(µ)']  = (activities_df['Max'] + 4*activities_df['+Probable'] + activities_df['Min']) / 6
activities_df['σ']          = (activities_df['Max'] - activities_df['Min']) / 6
activities_df['σ^2']        = activities_df['σ']**2
activities_df['α']          = ((activities_df['Media_(µ)'] - activities_df['Min']) / (activities_df['Max'] - activities_df['Min'])) * (((activities_df['Media_(µ)'] - activities_df['Min']) * (activities_df['Max'] - activities_df['Media_(µ)']) / activities_df['σ']**2) - 1)
activities_df['β']          = ((activities_df['Max'] - activities_df['Media_(µ)']) / (activities_df['Media_(µ)'] - activities_df['Min'])) * activities_df['α']

In [4]:
print(activities_df)

   N  Actividades  Min  +Probable    Max  Ruta_Critica  Media_(µ)         σ  \
0  1  Actividad 1   36         40   50.0             1      41.00  2.333333   
1  2  Actividad 2   27         30   37.5             0      30.75  1.750000   
2  3  Actividad 3   72         80  100.0             1      82.00  4.666667   
3  4  Actividad 4   45         50   62.5             0      51.25  2.916667   
4  5  Actividad 5   90        100  125.0             1     102.50  5.833333   
5  6  Actividad 6   63         70   87.5             0      71.75  4.083333   
6  7  Actividad 7   54         60   75.0             0      61.50  3.500000   
7  8  Actividad n   18         20   25.0             1      20.50  1.166667   

         σ^2         α         β  
0   5.444444  2.594752  4.670554  
1   3.062500  2.594752  4.670554  
2  21.777778  2.594752  4.670554  
3   8.506944  2.594752  4.670554  
4  34.027778  2.594752  4.670554  
5  16.673611  2.594752  4.670554  
6  12.250000  2.594752  4.670554  
7   1.36

Calculamos los valores de la Beta-Pert

In [15]:
dist_df = {
    '%_Probabilidad': [random.random() for _ in range(len(activities_df))],
}

dist_df['Beta-Pert'] = stats.beta.ppf(
    dist_df['%_Probabilidad'],
    activities_df['α'],
    activities_df['β'],
    activities_df['Min'],
    activities_df['Max'],
)

dist_df = pd.DataFrame(dist_df)

print(dist_df)

   %_Probabilidad   Beta-Pert
0        0.846805   62.924477
1        0.868626   47.889904
2        0.990839  148.930416
3        0.766624   75.093309
4        0.129426  110.704760
5        0.326265   86.205906
6        0.877687   96.401447
7        0.990518   37.186207
